In [28]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import random

In [6]:
data = pd.read_csv('../dataset/indian-male-names.csv')

In [17]:
indian_names = data['name']
names = indian_names.to_list()

In [22]:
mod_names = []

for name in names:
    if type(name) is str:
        mod_names.append(name)

len(mod_names), len(names)

(14821, 14845)

In [26]:
chars = sorted(list(set(''.join(mod_names))))
print(chars)
print(len(chars))

[' ', '&', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '@', '[', '\\', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ं', 'अ', 'आ', 'उ', 'ऐ', 'क', 'ख', 'ग', 'च', 'छ', 'ज', 'ण', 'त', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ो', '्', '\u200d', '�']
87


In [27]:
s_toi = {s:i+1 for i,s in enumerate(chars)}
s_toi['.'] = 0

i_tos = {i:s for s,i in s_toi.items()}
print(i_tos)

{1: ' ', 2: '&', 3: '(', 4: ')', 5: ',', 6: '-', 0: '.', 8: '/', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: '9', 19: '@', 20: '[', 21: '\\', 22: '`', 23: 'a', 24: 'b', 25: 'c', 26: 'd', 27: 'e', 28: 'f', 29: 'g', 30: 'h', 31: 'i', 32: 'j', 33: 'k', 34: 'l', 35: 'm', 36: 'n', 37: 'o', 38: 'p', 39: 'q', 40: 'r', 41: 's', 42: 't', 43: 'u', 44: 'v', 45: 'w', 46: 'x', 47: 'y', 48: 'z', 49: 'ं', 50: 'अ', 51: 'आ', 52: 'उ', 53: 'ऐ', 54: 'क', 55: 'ख', 56: 'ग', 57: 'च', 58: 'छ', 59: 'ज', 60: 'ण', 61: 'त', 62: 'द', 63: 'ध', 64: 'न', 65: 'प', 66: 'फ', 67: 'ब', 68: 'म', 69: 'य', 70: 'र', 71: 'ल', 72: 'व', 73: 'श', 74: 'ष', 75: 'स', 76: 'ह', 77: '़', 78: 'ा', 79: 'ि', 80: 'ी', 81: 'ु', 82: 'ू', 83: 'े', 84: 'ो', 85: '्', 86: '\u200d', 87: '�'}


In [30]:
def build_dataset(words):
    block_size = 3 # context length
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        
        for ch in w + '.':
            ix = s_toi[ch]
            X.append(context)
            Y.append(ix)
            # crop the first latter and append the one ahead
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)

    return X, Y

random.shuffle(mod_names)

n1 = int(0.8 * len(mod_names))
n2 = int(0.9 * len(mod_names))

Xtr, Ytr = build_dataset(mod_names[:n1])
X_dev, Y_dev = build_dataset(mod_names[n1:n2])
X_test, Y_test = build_dataset(mod_names[n2:])

In [31]:
Xtr.shape, X_dev.shape, X_test.shape

(torch.Size([131114, 3]), torch.Size([16315, 3]), torch.Size([16071, 3]))

In [37]:
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in ** 0.5
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        
        # parameters trained with backprop
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

        # buffers (trained with a running momentum update)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True) # batch mean
            xvar = x.var(0, keepdim=True) # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

In [38]:
n_embed = 10 # dimension of the character embedding vectors
n_hidden = 100 # number of neurons in the hidden layer of MLP
vocab_size = len(chars)
block_size = 3

C = torch.randn((vocab_size, n_embed))

layers = [
    Linear(n_embed * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]

with torch.no_grad():
    # last layer: make less confident
    layers[-1].gamma *= 0.1
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 1.0

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total

for p in parameters:
    p.requires_grad = True

53744


In [40]:
# same optimization as last time
max_steps = 10000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
  
  # mini-batch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  x = emb.view(emb.shape[0], -1) # concatenate the vectors
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function
  
  # backward pass
  for layer in layers:
    layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 # if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 2000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])


      0/  10000: 1.5937
   2000/  10000: 1.3623
   4000/  10000: 1.5735
   6000/  10000: 1.3002
   8000/  10000: 1.6642


In [41]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (X_dev, Y_dev),
    'test': (X_test, Y_test),
  }[split]

  emb = C[x] # (N, block_size, n_embd)
  x = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  
  for layer in layers:
    x = layer(x)

  loss = F.cross_entropy(x, y)
  print(split, loss.item())

# put layers into eval mode
for layer in layers:
  layer.training = False

split_loss('train')
split_loss('val')

train 1.5946717262268066
val 1.6427440643310547


In [42]:
# sample from the model
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      emb = C[torch.tensor([context])] # (1,block_size,n_embd)
      x = emb.view(emb.shape[0], -1) # concatenate the vectors
      
      for layer in layers:
        x = layer(x)
      
      logits = x
      probs = F.softmax(logits, dim=1)
      
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1).item()
      
      # shift the context window and track the samples
      context = context[1:] + [ix]
      out.append(ix)
      
      # if we sample the special '.' token, break
      if ix == 0:
        break
    
    print(''.join(i_tos[i] for i in out)) # decode and print the generated word

ratendeep.
firajtoom  kumar.
sarveen @ shanchand.
rajesh.
abhishan singh ram.
mेar5sh.
rauk\a s/o amithuri.
sarkash.
karti.
umar.
ayan kumar singh.
anishona.
mungy.
bhamit.
mohd.
mukesh.
ovindra singh singh.
chharma.
raj @ a9siya.
mahes@ pankit.
